# Setup

In [4]:

from pathlib import Path
import os
import pprint
import pandas as pd
import pygwalker as pyg
import dabl

import sys
from pathlib import Path
#### For Path setup
def setup_project_path():
    """프로젝트 루트 경로를 찾아서 파이썬 경로에 추가"""
    current = Path.cwd()
    while current != current.parent:
        if (current / '.git').exists():
            if str(current) not in sys.path:
                sys.path.append(str(current))
                print(f'Project root found: {current}')
            return current
        current = current.parent
    return None

# 프로젝트 경로 설정
project_root = setup_project_path()
if project_root is None:
    # 프로젝트 루트를 찾지 못했다면 직접 지정
    project_root = Path("D:/dev/upstageailab5-ml-regression-ml_r4")
    if str(project_root) not in sys.path:
        sys.path.append(str(project_root))

#jupyter nbextension enable --py widgetsnbextension

import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False


In [5]:
from src.logger import Logger
from src.preprocessing import DataPrep
from src.eda import EDA
from src.feature import FeatureEngineer, Clustering, XAI
from src.train import Model
from src.visualization import Visualizer
from src.utils import Utils, PathManager
## memory management
import gc
gc.collect()
########################################################################################################################################
logger_instance = Logger()
logger = logger_instance.logger
utils = Utils(logger)
utils.setup_font_and_path_platform()
current_platform = utils.current_platform
#os.environ['PYTHONPATH'] = r'D:\dev\upstageailab5-ml-regression-ml_r4'
current_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
logger.info(f'#### Current workspalce: {current_path}')
if current_platform == 'Windows':
    base_path = Path(r'D:\dev\upstageailab5-ml-regression-ml_r4')
    logger.info(f'{current_platform} platform. Path: {base_path}')
elif current_platform == 'Darwin':          # Mac
    base_path = Path('/data/ephemeral/home/dev/upstageailab5-ml-regression-ml_r4')
    logger.info(f'{current_platform} platform. Path: {base_path}')
else:
    base_path = Path('/data/ephemeral/home/dev/upstageailab5-ml-regression-ml_r4')    # Linux
    logger.info(f'{current_platform} platform. Path: {base_path}')
########################################################################################################################################

########################################################################################################################################
### Data Prep

pm = PathManager(base_path)
pm.config


2024-11-11 17:46:08,995 - root - INFO - Initialized Logger.
2024-11-11 17:46:08,997 - root - INFO - Windows platform. Font: ['Malgun Gothic']
2024-11-11 17:46:08,999 - root - INFO - #### Current workspalce: d:\dev\upstageailab5-ml-regression-ml_r4
2024-11-11 17:46:09,000 - root - INFO - Windows platform. Path: D:\dev\upstageailab5-ml-regression-ml_r4


{'name': {'dataset_name': 'baseline',
  'split_type': 'hold_out',
  'model_name': 'random_forest'},
 'target': 'target',
 'thr_ratio_outlier': 0.01,
 'thr_ratio_null': 0.9,
 'thr_detect_categorical': '1e-7',
 'thr_null': 1000000,
 'time_delay': 3,
 'random_seed': 2024,
 'model_params': {'n_estimators': 3000, 'max_depth': 5, 'learning_rate': 0.01},
 'data_paths': {'train': 'data/train.csv', 'test': 'data/test.csv'},
 'features': ['feature1', 'feature2', 'feature3'],
 'wandb': {'project': 'project-regression_house_price',
  'entity': 'joon',
  'group': 'group-ml4'},
 'sweep_configs': {'xgboost': {'method': 'bayes',
   'metric': {'name': 'rmse', 'goal': 'minimize'},
   'parameters': {'eta': 0.3,
    'max_depth': 10,
    'subsample': 0.6239,
    'colsample_bytree': 0.5305,
    'gamma': 4.717,
    'reg_lambda': 5.081,
    'alpha': 0.4902,
    'n_estimators': 551}},
  'random_forest': {'method': 'bayes',
   'metric': {'name': 'rmse', 'goal': 'minimize'},
   'parameters': {'n_estimators': 200

In [6]:
# PathManager에서 경로 가져오기
data_path = pm.get_path('data', as_str=True)
prep_path = pm.get_path('processed_path', as_str=True)
output_path = pm.get_path('output', as_str=True)

# 설정 딕셔너리 생성
config = {   
    'out_path': output_path,
    'base_path': str(pm.base_path),
    'data_paths': {
        'subway_feature': str(Path(data_path) / 'subway_feature.csv'),
        'bus_feature': str(Path(data_path) / 'bus_feature.csv'),
    },
    'prep_paths': {
        'baseline': str(Path(prep_path) / 'df_base_prep.csv'),
        'auto': str(Path(prep_path) / 'df_auto_prep.csv'),
        'feat': str(Path(prep_path) / 'df_feat.csv'),
        'feat_add': str(Path(prep_path) / 'df_feat_add.csv')
    },
    'logger': logger_instance
}

# 필요한 경우 개별 경로 접근
path_baseline = config['prep_paths']['baseline']
path_auto = config['prep_paths']['auto']
path_feat = config['prep_paths']['feat']
path_feat_add = config['prep_paths']['feat_add']

config.update(pm.paths)
pprint.pprint(config)

{'base_path': 'D:\\dev\\upstageailab5-ml-regression-ml_r4',
 'config': WindowsPath('D:/dev/upstageailab5-ml-regression-ml_r4/config'),
 'data': WindowsPath('D:/dev/upstageailab5-ml-regression-ml_r4/data'),
 'data_paths': {'bus_feature': 'D:\\dev\\upstageailab5-ml-regression-ml_r4\\data\\bus_feature.csv',
                'subway_feature': 'D:\\dev\\upstageailab5-ml-regression-ml_r4\\data\\subway_feature.csv'},
 'logger': <src.logger.Logger object at 0x000001C2C478DF10>,
 'logs': WindowsPath('D:/dev/upstageailab5-ml-regression-ml_r4/logs'),
 'models_path': WindowsPath('D:/dev/upstageailab5-ml-regression-ml_r4/output/models'),
 'out_path': 'D:\\dev\\upstageailab5-ml-regression-ml_r4\\output',
 'output': WindowsPath('D:/dev/upstageailab5-ml-regression-ml_r4/output'),
 'prep_paths': {'auto': 'D:\\dev\\upstageailab5-ml-regression-ml_r4\\data\\preprocessed\\df_auto_prep.csv',
                'baseline': 'D:\\dev\\upstageailab5-ml-regression-ml_r4\\data\\preprocessed\\df_base_prep.csv',
      

## Setup Path

In [ ]:
 ####
# column_types =Utils.detect_column_types(df_feat, unique_ratio_threshold=1e-7, exclude_cols=['is_test', 'target'])
# # 0.0000001 일때 is_numeric_dtype 과 비슷한 결과 나옴.
# categorical_features = column_types['categorical']
# numerical_features = column_types['numerical']   

In [135]:
## 데이터 불러오기
df_interpolation = pd.read_csv(os.path.join(prep_path, 'df_interpolation.csv'), index_col=0)
feat_cluster = pd.read_csv(os.path.join(prep_path, 'feat_cluster.csv'))
feat_transport = pd.read_csv(os.path.join(prep_path, 'feat_transport.csv'))
feat_baseline = pd.read_csv(os.path.join(prep_path, 'feat_baseline.csv'))

cols_feat_baseline = feat_baseline.columns
cols_feat_cluster = feat_cluster.columns
cols_feat_transport = feat_transport.columns

In [136]:
print(f'{len(cols_feat_baseline)} {cols_feat_baseline}\n{len(cols_feat_cluster)} {cols_feat_cluster}\n{len(cols_feat_transport)} {cols_feat_transport}')


6 Index(['구', '동', '계약년', '계약월', '강남여부', '신축여부'], dtype='object')
4 Index(['cluster_dist_transport', 'cluster_dist_transport_count',
       'cluster_select', 'cluster_select_count'],
      dtype='object')
10 Index(['subway_station_area_count', 'subway_direct_influence_count',
       'subway_indirect_influence_count', 'subway_shortest_distance',
       'subway_zone_type', 'bus_station_area_count',
       'bus_direct_influence_count', 'bus_indirect_influence_count',
       'bus_shortest_distance', 'bus_zone_type'],
      dtype='object')


In [137]:

df_new = pd.concat(
    [
        df_interpolation.reset_index(drop=True),
        feat_baseline.reset_index(drop=True),
        feat_cluster.reset_index(drop=True),
        feat_transport.reset_index(drop=True)
    ],
    axis=1
)

cols_to_drop = ['시군구','계약년월']
df_new.drop(columns=cols_to_drop, inplace=True)


In [166]:

################# Feature Categorical & Numerical
cateogirical_features_add = ['강남여부', '신축여부','구', '동']
categorical_features = ['k-건설사(시공사)', 'k-관리방식', 'k-난방방식', 'k-단지분류(아파트,주상복합등등)', 'k-복도유형', 'k-사용검사일-사용승인일', 'k-세대타입(분양형태)', 'k-수정일자', 'k-시행사', '경비비관리형태', '관리비 업로드', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '단지신청일', '도로명', '번지', '사용허가여부', '세대전기계약방법',  '아파트명', '청소비관리형태']+cateogirical_features_add

# 시군구 제외
numerical_features_add = ['계약년', '계약월']
numerical_features = ['k-85㎡~135㎡이하', 'k-관리비부과면적', 'k-연면적', 'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-전용면적별세대현황(60㎡이하)', 'k-전체동수', 'k-전체세대수', 'k-주거전용면적', '건축년도', '건축면적', '계약일', '본번', '부번', '전용면적', '좌표X', '좌표Y', '주차대수', '층']+ numerical_features_add
# 계약년월 제외
print(len(categorical_features), len(numerical_features))
print(f'feat_transport.columns: {feat_transport.columns}, feat_cluster.columns: {feat_cluster.columns}')

numerical_transport_features=['subway_shortest_distance', 'bus_shortest_distance']
categorical_features += list(feat_cluster.columns) + list(set(feat_transport.columns) -set(numerical_transport_features))
numerical_features += numerical_transport_features
print(f'categorical_features: {len(categorical_features)}, numerical_features: {len(numerical_features)}')

list_to_chk = categorical_features + numerical_features

remaining_columns = [col for col in df_new.columns if col not in list_to_chk]
print(f'remaining_columns: {remaining_columns}')


assert len(categorical_features) + len(numerical_features) +2== df_new.shape[1]

24 20
feat_transport.columns: Index(['subway_station_area_count', 'subway_direct_influence_count',
       'subway_indirect_influence_count', 'subway_shortest_distance',
       'subway_zone_type', 'bus_station_area_count',
       'bus_direct_influence_count', 'bus_indirect_influence_count',
       'bus_shortest_distance', 'bus_zone_type'],
      dtype='object'), feat_cluster.columns: Index(['cluster_dist_transport', 'cluster_dist_transport_count',
       'cluster_select', 'cluster_select_count'],
      dtype='object')
categorical_features: 36, numerical_features: 22
remaining_columns: ['target', 'is_test']


In [133]:
df_ff= pd.read_csv(os.path.join(prep_path, 'df_feature.csv'))
df_ff.shape

(1128094, 46)

In [141]:
df_new.to_csv(os.path.join(prep_path, 'df_feature_add.csv'), index=False)

In [168]:
df_new.isnull().sum()


번지                                 0
본번                                 0
부번                                 0
아파트명                               0
전용면적                               0
계약일                                0
층                                  0
건축년도                               0
도로명                                0
k-단지분류(아파트,주상복합등등)                 0
k-세대타입(분양형태)                       0
k-관리방식                             0
k-복도유형                             0
k-난방방식                             0
k-전체동수                             0
k-전체세대수                            0
k-건설사(시공사)                         0
k-시행사                              0
k-사용검사일-사용승인일                      0
k-연면적                              0
k-주거전용면적                           0
k-관리비부과면적                          0
k-전용면적별세대현황(60㎡이하)                 0
k-전용면적별세대현황(60㎡~85㎡이하)             0
k-85㎡~135㎡이하                       0
k-수정일자                             0
경비비관리형태                            0
세

# Scaling Numerical Features

In [142]:
# from src.utils import Utils
# from typing import List, Dict
# from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer
# from sklearn.pipeline import Pipeline
# from tqdm import tqdm

# class DataScaler:
#     def __init__(self, 
#                  target_col: str,
#                  categorical_cols: List[str] = None,
#                  continuous_cols: List[str] = None,
#                  exclude_cols: List[str] = None):
#         self.target_col = target_col
#         self.categorical_cols = set(categorical_cols) if categorical_cols else set()
#         self.continuous_cols = set(continuous_cols) if continuous_cols else set()
#         self.exclude_cols = set(exclude_cols) if exclude_cols else set()
#         self.scalers = {}
        
#     def _get_appropriate_scaler(self, col_name: str, data: pd.Series) -> object:
#         """컬럼 특성에 맞는 스케일러 반환"""
#         if col_name in self.categorical_cols:
#             return StandardScaler()
            
#         if abs(data.skew()) > 1:
#             return Pipeline([
#                 ('robust', RobustScaler()),
#                 ('power', PowerTransformer(method='yeo-johnson'))
#             ])
        
#         return RobustScaler()

#     def scale_features(self, df: pd.DataFrame, is_train: bool = True) -> pd.DataFrame:
#         """특성별 적절한 스케일링 적용"""
#         scaled_df = df.copy()

#         # 스케일링할 컬럼 결정
#         if self.continuous_cols:
#             scale_cols = self.continuous_cols | self.categorical_cols
#         else:
#             scale_cols = set(df.select_dtypes(include=['int64', 'float64']).columns)
        
#         # 제외할 컬럼 제거
#         scale_cols = scale_cols - self.exclude_cols - {self.target_col, 'is_test'}
        
#         # 누락된 컬럼 확인
#         missing_cols = scale_cols - set(df.columns)
#         if missing_cols:
#             print(f"Warning: 다음 컬럼들이 데이터에 없습니다: {missing_cols}")
        
#         # 실제 스케일링할 컬럼
#         actual_scale_cols = scale_cols & set(df.columns)
#         print(f"스케일링 적용 컬럼: {sorted(actual_scale_cols)}")
        
#         for col in tqdm(actual_scale_cols):
#             if is_train:
#                 scaler = self._get_appropriate_scaler(col, df[col])
#                 self.scalers[col] = scaler
#                 scaled_values = scaler.fit_transform(df[[col]]).ravel()  # 1D로 변환
#                 scaled_df[col] = scaled_values
#             else:
#                 if col in self.scalers:
#                     scaled_values = self.scalers[col].transform(df[[col]]).ravel()  # 1D로 변환
#                     scaled_df[col] = scaled_values
                
#         return scaled_df


In [187]:
from typing import List, Dict
from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer
from sklearn.pipeline import Pipeline
from tqdm import tqdm

class DataScaler:
    def __init__(self, 
              
                 categorical_cols: List[str] = None,
                 continuous_cols: List[str] = None,
                 exclude_cols: List[str] = None):
        """
        데이터 스케일링을 위한 클래스
        
        Args:
            target_col: 타겟 컬럼명 (스케일링에서 제외)
            categorical_cols: 범주형 변수 리스트
            continuous_cols: 연속형 변수 리스트
            exclude_cols: 스케일링에서 제외할 컬럼 리스트
        """
     
        self.categorical_cols = set(categorical_cols) if categorical_cols else set()
        self.continuous_cols = set(continuous_cols) if continuous_cols else set()
        self.exclude_cols = set(exclude_cols) if exclude_cols else set()
        self.scalers = {}
        
    def _get_appropriate_scaler(self, col_name: str, data: pd.Series) -> object:
        """
        컬럼 특성에 맞는 스케일러 반환
        
        Args:
            col_name: 컬럼명
            data: 스케일링할 데이터
            
        Returns:
            스케일러 객체
        """
        # 범주형 변수는 StandardScaler 사용
        if col_name in self.categorical_cols:
            return StandardScaler()
        
        # 연속형 변수는 왜도에 따라 스케일러 결정
        if abs(data.skew()) > 1:
            # 심한 왜도는 RobustScaler + PowerTransformer
            return Pipeline([
                ('robust', RobustScaler()),
                ('power', PowerTransformer(method='yeo-johnson'))
            ])
        
        # 나머지는 RobustScaler
        return RobustScaler()
        
    def scale_features(self, df: pd.DataFrame, is_train: bool = True) -> pd.DataFrame:
        """
        특성별 적절한 스케일링 적용
        
        Args:
            df: 스케일링할 데이터프레임
            is_train: 학습 데이터 여부
            
        Returns:
            스케일링된 데이터프레임
        """
        scaled_df = df.copy()
        
        # 스케일링할 컬럼 결정
        if self.continuous_cols or self.categorical_cols:
            # 지정된 컬럼이 있는 경우
            scale_cols = self.continuous_cols | self.categorical_cols
        else:
            # 지정된 컬럼이 없는 경우 수치형 컬럼 자동 선택
            scale_cols = set(df.select_dtypes(include=['int64', 'float64']).columns)
        
        # 제외할 컬럼 처리
        exclude_set = self.exclude_cols.copy()
        exclude_set.add('is_test')  # 항상 제외
        
        scale_cols = scale_cols - exclude_set
        
        # 실제 존재하는 컬럼만 선택
        actual_scale_cols = scale_cols & set(df.columns)
        
        # 누락된 컬럼 확인
        missing_cols = scale_cols - actual_scale_cols
        if missing_cols:
            print(f"Warning: 다음 컬럼들이 데이터에 없습니다: {sorted(missing_cols)}")
        
        print(f"스케일링 적용 컬럼: {sorted(actual_scale_cols)}")
        
        # 스케일링 수행
        for col in tqdm(actual_scale_cols, desc='Scaling columns'):
            if is_train:
                # 학습 데이터: 스케일러 생성 및 적용
                scaler = self._get_appropriate_scaler(col, scaled_df[col])
                self.scalers[col] = scaler
                scaled_values = scaler.fit_transform(scaled_df[[col]]).ravel()
                scaled_df[col] = scaled_values
            else:
                # 테스트 데이터: 기존 스케일러 적용
                if col in self.scalers:
                    scaled_values = self.scalers[col].transform(scaled_df[[col]]).ravel()
                    scaled_df[col] = scaled_values
                else:
                    print(f"Warning: {col} 컬럼의 스케일러가 없습니다.")
        
        return scaled_df

In [ ]:
def unconcat_train_test(concat):
    Utils.remove_unnamed_columns(concat)
    dt = concat.query('is_test==0')
    # y_train = dt['target']
    dt.drop(columns=['is_test'], inplace=True)
    dt_test = concat.query('is_test==1')
    dt_test.drop(columns=['is_test'], inplace=True)
    return dt, dt_test

In [197]:
df_train, df_test = Utils.unconcat_train_test(df_new) # train, test data 분리

# # categorical, numerical 기준은 baseline 으로 임시 작성. 변경 가능.
# categorical_features =['전용면적', '계약일', '층', '건축년도', 'k-전체동수', 'k-전체세대수', 'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)', 'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', '건축면적', '주차대수', '좌표X', '좌표Y', 'target', '강남여부', '신축여부']
# numerical_features = ['번지', '본번', '부번', '아파트명', '도로명', 'k-단지분류(아파트,주상복합등등)', 'k-전화번호', 'k-팩스번호', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형', 'k-난방방식', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드', '단지신청일', '구', '동', '계약년', '계약월']
# add_features = ['구', '동', '계약년', '계약월']
# numerical_features = list(set(numerical_features)-set(add_features))

data_scaler = DataScaler(
                            categorical_cols=categorical_features,
                            continuous_cols=numerical_features,
                            exclude_cols=['target']) # 아직 

# 1. Numerical Features 들의 경우, outlier removal 없이 이상치에 강건하게 Robust Scaling 을 먼저 적용합니다.
# 학습 데이터 전처리
train_scaled = data_scaler.scale_features(df_train[numerical_features], is_train=True)
# 테스트 데이터 전처리 (학습 데이터의 스케일러 사용)
test_scaled = data_scaler.scale_features(df_test[numerical_features], is_train=False)


# 2. Categorical Features 들의 경우, Feature engineering 후 encoding 까지 완료된 이후에 표준화 스케일링을 별도 적용합니다
print(train_scaled.isnull().sum(), test_scaled.isnull().sum())
print(train_scaled.shape, test_scaled.shape)


스케일링 적용 컬럼: ['bus_shortest_distance', 'k-85㎡~135㎡이하', 'k-관리비부과면적', 'k-연면적', 'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-전용면적별세대현황(60㎡이하)', 'k-전체동수', 'k-전체세대수', 'k-주거전용면적', 'subway_shortest_distance', '건축년도', '건축면적', '계약년', '계약월', '계약일', '본번', '부번', '전용면적', '좌표X', '좌표Y', '주차대수', '층']


Scaling columns: 100%|██████████| 22/22 [00:45<00:00,  2.05s/it]


스케일링 적용 컬럼: ['bus_shortest_distance', 'k-85㎡~135㎡이하', 'k-관리비부과면적', 'k-연면적', 'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-전용면적별세대현황(60㎡이하)', 'k-전체동수', 'k-전체세대수', 'k-주거전용면적', 'subway_shortest_distance', '건축년도', '건축면적', '계약년', '계약월', '계약일', '본번', '부번', '전용면적', '좌표X', '좌표Y', '주차대수', '층']


Scaling columns: 100%|██████████| 22/22 [00:00<00:00, 200.00it/s]

k-85㎡~135㎡이하                0
k-관리비부과면적                   0
k-연면적                       0
k-전용면적별세대현황(60㎡~85㎡이하)      0
k-전용면적별세대현황(60㎡이하)          0
k-전체동수                      0
k-전체세대수                     0
k-주거전용면적                    0
건축년도                        0
건축면적                        0
계약일                         0
본번                          0
부번                          0
전용면적                        0
좌표X                         0
좌표Y                         0
주차대수                        0
층                           0
계약년                         0
계약월                         0
subway_shortest_distance    0
bus_shortest_distance       0
dtype: int64 k-85㎡~135㎡이하                0
k-관리비부과면적                   0
k-연면적                       0
k-전용면적별세대현황(60㎡~85㎡이하)      0
k-전용면적별세대현황(60㎡이하)          0
k-전체동수                      0
k-전체세대수                     0
k-주거전용면적                    0
건축년도                        0
건축면적                        0
계약일                        

In [204]:
train_scaled['target'] = df_train['target']

train_scaled[categorical_features] = df_train[categorical_features]
test_scaled[categorical_features] = df_test[categorical_features]

concat_scaled = Utils.concat_train_test(train_scaled, test_scaled)
concat_scaled.to_csv(os.path.join(prep_path, 'df_scaled.csv'), index=False)

0    1118822
1       9272
Name: is_test, dtype: int64


In [211]:
concat_scaled.isnull().sum()


k-85㎡~135㎡이하                       0
k-관리비부과면적                          0
k-연면적                              0
k-전용면적별세대현황(60㎡~85㎡이하)             0
k-전용면적별세대현황(60㎡이하)                 0
k-전체동수                             0
k-전체세대수                            0
k-주거전용면적                           0
건축년도                               0
건축면적                               0
계약일                                0
본번                                 0
부번                                 0
전용면적                               0
좌표X                                0
좌표Y                                0
주차대수                               0
층                                  0
계약년                                0
계약월                                0
subway_shortest_distance           0
bus_shortest_distance              0
k-건설사(시공사)                         0
k-관리방식                             0
k-난방방식                             0
k-단지분류(아파트,주상복합등등)                 0
k-복도유형                             0
k

# Label Encoding for Categorical Features

In [206]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
def encode_label(dt_train, dt_test, categorical_columns_v2):
    logger.info('#### 범주형 변수들을 대상으로 레이블인코딩을 진행해 주겠습니다.')
    # 각 변수에 대한 LabelEncoder를 저장할 딕셔너리
    label_encoders = {}

    # Implement Label Encoding
    for col in tqdm( categorical_columns_v2 ):
        lbl = LabelEncoder()
    
        # Label-Encoding을 fit
        lbl.fit( dt_train[col].astype(str) )
        dt_train[col] = lbl.transform(dt_train[col].astype(str))
        label_encoders[col] = lbl           # 나중에 후처리를 위해 레이블인코더를 저장해주겠습니다.

        # Test 데이터에만 존재하는 새로 출현한 데이터를 신규 클래스로 추가해줍니다.
        dt_test[col] = dt_test[col].astype(str)
        for label in np.unique(dt_test[col]):
            if label not in lbl.classes_: # unseen label 데이터인 경우
                lbl.classes_ = np.append(lbl.classes_, label) # 미처리 시 ValueError발생하니 주의하세요!
        dt_test[col] = lbl.transform(dt_test[col].astype(str))

        dt_train.head(1)        # 레이블인코딩이 된 모습입니다.

        assert dt_train.shape[1] == dt_test.shape[1]          # train/test dataset의 shape이 같은지 확인해주겠습니다.
    return dt_train, dt_test, label_encoders

In [207]:
train_encoded, test_encoded, label_encoders = encode_label(df_train[categorical_features], df_test[categorical_features] ,categorical_features)


2024-11-11 20:07:08,493 - root - INFO - #### 범주형 변수들을 대상으로 레이블인코딩을 진행해 주겠습니다.


100%|██████████| 36/36 [00:28<00:00,  1.28it/s]


In [212]:
train_encoded[numerical_features] = df_train[numerical_features]
test_encoded[numerical_features] = df_test[numerical_features]
train_encoded['target'] = df_train['target']


In [213]:
train_encoded.shape, test_encoded.shape

df_encoded = Utils.concat_train_test(train_encoded, test_encoded)
df_encoded.to_csv(os.path.join(prep_path, 'df_encoded.csv'), index=False)

df_encoded.isnull().sum()

0    1118822
1       9272
Name: is_test, dtype: int64


In [ ]:
#prep_path = os.path.join(base_path, 'data', 'preprocessed') # base_path 는 프로젝트 루트 경로 등
#df_feat = pd.read_csv(os.path.join(prep_path, 'df_feature.csv'), index_col=0) # train, test data 가 합쳐진 데이터 불러오기; is_test =0, 1 로 구분

####
column_types =Utils.detect_column_types(df_new, unique_ratio_threshold=1e-5, exclude_cols=['is_test', 'target'])
# 0.0000001 일때 is_numeric_dtype 과 비슷한 결과 나옴.
categorical_features_1 = column_types['categorical']
numerical_features_1 = column_types['numerical']   

In [112]:
def prep_feat(concat):
        
    concat['구'] = concat['시군구'].map(lambda x : x.split()[1])
    concat['동'] = concat['시군구'].map(lambda x : x.split()[2])
    del concat['시군구']

    concat['계약년'] = concat['계약년월'].astype('str').map(lambda x : x[:4])
    concat['계약월'] = concat['계약년월'].astype('str').map(lambda x : x[4:])
    del concat['계약년월']

    all = list(concat['구'].unique())
    gangnam = ['강서구', '영등포구', '동작구', '서초구', '강남구', '송파구', '강동구']
    gangbuk = [x for x in all if x not in gangnam]

    assert len(all) == len(gangnam) + len(gangbuk)       # 알맞게 분리되었는지 체크합니다.

    # 강남의 여부를 체크합니다.
    is_gangnam = []
    for x in concat['구'].tolist() :
        if x in gangnam :
            is_gangnam.append(1)
        else :
            is_gangnam.append(0)

    # 파생변수를 하나 만릅니다.
    concat['강남여부'] = is_gangnam
    # 건축년도 분포는 아래와 같습니다. 특히 2005년이 Q3에 해당합니다.
    # 2009년 이후에 지어진 건물은 10%정도 되는 것을 확인할 수 있습니다.
    concat['건축년도'].describe(percentiles = [0.1, 0.25, 0.5, 0.75, 0.8, 0.9])
   
    concat['신축여부'] = concat['건축년도'].apply(lambda x: 1 if x >= 2020 else 0) # 2009
    cols_feat = ['구', '동','계약년', '계약월','강남여부','신축여부']
    concat[cols_feat].to_csv(os.path.join(prep_path, 'feat_baseline.csv'), index=False)
    concat.head(1)       # 최종 데이터셋은 아래와 같습니다.
    
    return concat

In [116]:
prep_feat(df_interpolation)


,번지,본번,부번,아파트명,전용면적,계약일,층,건축년도,도로명,"k-단지분류(아파트,주상복합등등)",...,좌표Y,단지신청일,target,is_test,구,동,계약년,계약월,강남여부,신축여부
0,658-1,658.0,1.0,개포6차우성,79.97,8,3,1987,언주로 3,아파트,...,37.476763,2022-11-17 10:19:06.0,124000.000000,0,강남구,개포동,2017,12,1,0
1,658-1,658.0,1.0,개포6차우성,79.97,22,4,1987,언주로 3,아파트,...,37.476763,2022-11-17 10:19:06.0,123500.000000,0,강남구,개포동,2017,12,1,0
2,658-1,658.0,1.0,개포6차우성,54.98,28,5,1987,언주로 3,아파트,...,37.476763,2022-11-17 10:19:06.0,91500.000000,0,강남구,개포동,2017,12,1,0
3,658-1,658.0,1.0,개포6차우성,79.97,3,4,1987,언주로 3,아파트,...,37.476763,2022-11-17 10:19:06.0,130000.000000,0,강남구,개포동,2018,01,1,0
4,658-1,658.0,1.0,개포6차우성,79.97,8,2,1987,언주로 3,아파트,...,37.476763,2022-11-17 10:19:06.0,117000.000000,0,강남구,개포동,2018,01,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9267,816,816.0,0.0,신내우디안1단지,84.65,19,13,2014,신내역로1길 85,아파트,...,37.618870,2014-09-01 13:05:03.0,56449.488263,1,중랑구,신내동,2023,07,0,0
9268,816,816.0,0.0,신내우디안1단지,84.62,25,12,2014,신내역로1길 85,아파트,...,37.618870,2014-09-01 13:05:03.0,56449.488263,1,중랑구,신내동,2023,07,0,0
9269,816,816.0,0.0,신내우디안1단지,101.65,27,12,2014,신내역로1길 85,아파트,...,37.618870,2014-09-01 13:05:03.0,56449.488263,1,중랑구,신내동,2023,08,0,0
9270,816,816.0,0.0,신내우디안1단지,84.94,2,18,2014,신내역로1길 85,아파트,...,37.618870,2014-09-01 13:05:03.0,56449.488263,1,중랑구,신내동,2023,09,0,0


In [113]:
df_interpolation.columns


Index(['번지', '본번', '부번', '아파트명', '전용면적', '계약일', '층', '건축년도', '도로명',
       'k-단지분류(아파트,주상복합등등)', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형', 'k-난방방식',
       'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일', 'k-연면적',
       'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)', 'k-전용면적별세대현황(60㎡~85㎡이하)',
       'k-85㎡~135㎡이하', 'k-수정일자', '경비비관리형태', '세대전기계약방법', '청소비관리형태', '건축면적',
       '주차대수', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드', '좌표X',
       '좌표Y', '단지신청일', 'target', 'is_test'],
      dtype='object')

In [ ]:
import pickle
with open(os.path.join(prep_path, 'feat_engineering.pickle'), 'rb') as f:
    feat_eng = pickle.load(f)
categorical_columns_v2 = feat_eng.get('categorical_columns_v2')
continuous_columns_v2 = feat_eng.get('continuous_columns_v2')

print(f'categorical_columns_v2: {len(categorical_columns_v2)} {categorical_columns_v2}\n continuous_columns_v2: {len(continuous_columns_v2)} {continuous_columns_v2}')